# 02_Data_Preprocessing.ipynb

## Descripción General
Este cuaderno implementa el pipeline de ingeniería de datos (ETL) necesario para preparar el dataset CIFAR-10 para el entrenamiento de modelos SOTA.

## Objetivos
1. **Extracción:** Descargar el dataset CIFAR-10 oficial.
2. **Transformación:**
   - Filtrar únicamente las clases de interés: *Dog, Automobile, Bird*.
   - Normalizar tipos de datos a `float32` para compatibilidad con TensorFlow 2.x.
   - Realizar codificación One-Hot manual para asegurar el orden de clases.
   - Dividir los datos en conjuntos de Entrenamiento (Train) y Validación (Val).
3. **Carga (Persistencia):** Guardar los tensores procesados en formato `.npy` para maximizar la velocidad de carga en el entrenamiento.

## Configuración del Pipeline
- **Input:** CIFAR-10 (TensorFlow Datasets).
- **Output:** Archivos `.npy` (X_train, y_train, X_val, y_val, X_test, y_test).
- **Resolución:** 224x224 (Escalado posterior).

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm.auto import tqdm

# Verificacion de hardware disponible
print(f"Dispositivos GPU detectados: {tf.config.list_physical_devices('GPU')}")

# ============================================================================
# 0. CONFIGURACION GLOBAL
# ============================================================================

CONFIG = {
    'IMG_SIZE': 224,        # Resolucion objetivo para modelos ViT/ConvNeXt
    'BATCH_SIZE': 32,       # Tamano de lote estandar
    'NUM_CLASSES': 3,       # Clases activas
    'SEED': 42,             # Semilla para reproducibilidad
    'VALIDATION_SPLIT': 0.2 # 20% para validacion
}

# Mapeo de indices originales de CIFAR-10
# 1: Automobile, 2: Bird, 5: Dog
CLASS_MAPPING = {
    'dog': 5,
    'automobile': 1,
    'bird': 2,
}

# Orden estricto de las clases para el entrenamiento
CLASS_NAMES = ['dog', 'automobile', 'bird']

# Creacion de estructura de directorios
os.makedirs('/tf/notebooks/data', exist_ok=True)
os.makedirs('/tf/notebooks/outputs', exist_ok=True)

# Persistencia de la configuracion
with open('/tf/notebooks/config.json', 'w') as f:
    json.dump(CONFIG, f, indent=4)

print("Configuracion del sistema inicializada y guardada.")

## 1. Extracción de Datos
Se descarga el dataset CIFAR-10 completo desde los repositorios de TensorFlow Datasets.

In [ ]:
# ============================================================================
# 1. CARGA DE DATOS (SOURCE)
# ============================================================================

print("Iniciando descarga de CIFAR-10...")

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True,
    shuffle_files=True
)

print("Descarga completada exitosamente.")
print(f"Registros de Entrenamiento: {ds_info.splits['train'].num_examples}")
print(f"Registros de Prueba: {ds_info.splits['test'].num_examples}")

## 2. Filtrado y Transformación
En esta etapa se aplica la lógica de negocio para seleccionar solo las clases requeridas.
Se realiza un casteo explícito a `float32` para evitar conflictos de tipos durante operaciones matriciales complejas (como MixUp) en TensorFlow.

In [ ]:
# ============================================================================
# 2. PROCESAMIENTO Y FILTRADO
# ============================================================================

def extract_filtered_data(dataset, class_ids):
    """
    Filtra el dataset para conservar solo las clases especificas,
    aplica One-Hot Encoding manual y asegura tipos float32.

    Args:
        dataset: Objeto TFDS.
        class_ids: Lista de enteros con los IDs de CIFAR-10 a conservar.

    Returns:
        np.array: Imagenes filtradas (uint8).
        np.array: Etiquetas One-Hot (float32).
    """
    images = []
    labels = []
    
    print("Procesando particion de datos...")
    
    # Iteracion sobre el dataset para filtrado manual
    for img, label in tqdm(dataset, desc="Filtrando Clases"):
        label_val = label.numpy()
        
        if label_val in class_ids:
            # Las imagenes se mantienen en uint8 para optimizar almacenamiento en disco
            images.append(img.numpy()) 
            
            # One-hot encoding manual para garantizar el orden estricto:
            # [Dog, Automobile, Bird]
            if label_val == CLASS_MAPPING['dog']:
                labels.append([1., 0., 0.])
            elif label_val == CLASS_MAPPING['automobile']:
                labels.append([0., 1., 0.])
            elif label_val == CLASS_MAPPING['bird']:
                labels.append([0., 0., 1.])
    
    # Casting critico a float32 para las etiquetas
    return np.array(images), np.array(labels, dtype=np.float32)

# Ejecucion del filtrado
target_class_ids = list(CLASS_MAPPING.values())

print("Iniciando filtrado de Training Set...")
X_train_full, y_train_full = extract_filtered_data(ds_train, target_class_ids)

print("Iniciando filtrado de Test Set...")
X_test, y_test = extract_filtered_data(ds_test, target_class_ids)

print("\nResumen de dimensiones post-procesamiento:")
print(f"X_train shape: {X_train_full.shape} (Dtype: {X_train_full.dtype})")
print(f"y_train shape: {y_train_full.shape} (Dtype: {y_train_full.dtype})")

## 3. Verificación de Calidad de Datos (QA)
Se realiza una inspección visual rápida y un conteo de distribución de clases para asegurar que el proceso de filtrado no ha introducido sesgos o errores de etiquetado.

In [ ]:
# ============================================================================
# 3. VERIFICACION DE DATOS (QA)
# ============================================================================

# Calculo de distribucion de clases
train_counts = y_train_full.sum(axis=0)
test_counts = y_test.sum(axis=0)

print("Distribucion de clases resultante:")
print(f"Train: {dict(zip(CLASS_NAMES, train_counts))}")
print(f"Test:  {dict(zip(CLASS_NAMES, test_counts))}")

# Generacion de grilla de muestras para inspeccion visual
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
fig.suptitle('Muestras del Dataset Filtrado (Verificacion)', fontsize=16)

for idx in range(15):
    ax = axes[idx // 5, idx % 5]
    ax.imshow(X_train_full[idx])
    # Decodificacion de One-Hot a nombre de clase
    class_name = CLASS_NAMES[np.argmax(y_train_full[idx])]
    ax.set_title(f"{class_name}", fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.savefig('/tf/notebooks/outputs/01_dataset_qa_samples.png', dpi=150)
print("Imagen de verificacion guardada en outputs/01_dataset_qa_samples.png")
plt.show()

## 4. División del Dataset
Separación del conjunto de entrenamiento en **Entrenamiento** y **Validación** para monitorear el rendimiento del modelo y evitar el sobreajuste (overfitting). Se utiliza una semilla fija (`SEED=42`) para garantizar la reproducibilidad de los experimentos.

In [ ]:
# ============================================================================
# 4. SPLIT DE ENTRENAMIENTO Y VALIDACION
# ============================================================================

# Calculo de indices de corte
val_count = int(len(X_train_full) * CONFIG['VALIDATION_SPLIT'])
train_count = len(X_train_full) - val_count

# Mezcla aleatoria de indices (Shuffling)
np.random.seed(CONFIG['SEED'])
indices = np.random.permutation(len(X_train_full))

train_indices = indices[:train_count]
val_indices = indices[train_count:]

# Asignacion de datos basada en indices
X_train = X_train_full[train_indices]
y_train = y_train_full[train_indices]

X_val = X_train_full[val_indices]
y_val = y_train_full[val_indices]

print("Split completado:")
print(f"Set de Entrenamiento: {len(X_train)} muestras")
print(f"Set de Validacion:    {len(X_val)} muestras")
print(f"Set de Prueba:        {len(X_test)} muestras")

## 5. Almacenamiento (Persistencia)
Guardado de los arrays procesados en formato binario `.npy`. Este formato es altamente eficiente para lectura/escritura en NumPy y TensorFlow, reduciendo el tiempo de carga en el cuaderno de entrenamiento.

In [ ]:
# ============================================================================
# 5. PERSISTENCIA DE DATOS
# ============================================================================

print("Guardando datasets procesados en disco...")

# Guardado en formato NumPy (.npy)
np.save('/tf/notebooks/data/X_train.npy', X_train)
np.save('/tf/notebooks/data/y_train.npy', y_train)

np.save('/tf/notebooks/data/X_val.npy', X_val)
np.save('/tf/notebooks/data/y_val.npy', y_val)

np.save('/tf/notebooks/data/X_test.npy', X_test)
np.save('/tf/notebooks/data/y_test.npy', y_test)

# Guardado de metadatos del dataset para referencia
dataset_metadata = {
    'train_samples': len(X_train),
    'val_samples': len(X_val),
    'test_samples': len(X_test),
    'class_distribution': {
        'train': train_counts.tolist(),
        'test': test_counts.tolist()
    },
    'data_specs': {
        'X_dtype': str(X_train.dtype),
        'y_dtype': str(y_train.dtype)
    },
    **CONFIG
}

with open('/tf/notebooks/data/dataset_metadata.json', 'w') as f:
    json.dump(dataset_metadata, f, indent=4)

print("Pipeline finalizado exitosamente.")
print("Archivos generados en: /tf/notebooks/data/")